# Trabalho prático AC

## Trabalho realizado por:

Margarida Vila-chã, 202107923

Miguel Duarte Silva, 20210504

Sebastião Lessa, 202103238


## Algoritmo escolhido e breve explicação:

O algoritmo escolhido pelo nosso grupo foi o Random Forrest.

Ele usa uma combinação de árvores de decisão aleatórias. O processo começa com a criação de um número de árvores (floresta), em que cada uma das árvores criadas é treinada com uma amostra diferente. Isto permite a criação de árvores suficientemente diferentes, o que leva a que elas juntas consigam lidar com uma ampla variedade de situações, reduzindo o risco de overffitting.

Baseamos-nos nos seguintes data-sets:

- [breast-w](https://www.openml.org/search?type=run&id=7413&collections.id=99&run_task.task_id=15)
- [qsar-biodeg](https://www.openml.org/search?type=run&id=519587&collections.id=99&run_task.task_id=9957)
- [diabetes](https://www.openml.org/search?type=run&id=548&collections.id=99&run_task.task_id=37)


## Alteração escolhida
A alteração que o nosso grupo decidiu fazer no codigo fonte obtido foi considerar utilizar outros tipos de critérios. A nossa intenção é testar os 3 critérios possiveis e escolher como Classificador Random Forest final o que tiver mais precisão. Implementamos também uma forma de testar os 3 critérios em todas as Decision Trees e usar sempre a mais precisa na criação do Classificador Random Forest.


## Import das bibliotecas necessárias:

In [17]:
from collections import Counter
from time import time
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Modelo implementado, sem alteração

In [18]:
class DecisionTree:

    # class constructor
    def __init__(self, criterion='gini', max_depth=None):
        self.criterion = criterion
        self.max_depth = max_depth
        self.tree = None

    # method to train the tree
    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    # recursive method to build the tree based on training
    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        label_counts = Counter(y)

        # Base cases: check termination conditions
        if len(label_counts) == 1:
            # All samples belong to the same class
            return {'class': label_counts.most_common(1)[0][0]}

        if depth == self.max_depth or num_samples < 2:
            # Reached maximum depth or too few samples remaining
            return {'class': label_counts.most_common(1)[0][0]}

        if num_features == 0:
            # No features remaining to split on
            return {'class': label_counts.most_common(1)[0][0]}

        # Determine the best feature and split point
        best_feature, best_split_point = self._get_best_split(X, y)

        # Create the tree node
        node = {'feature': best_feature, 'split_point': best_split_point}

        # Split the data based on the best feature and split point
        left_mask = X[:, best_feature] <= best_split_point
        right_mask = X[:, best_feature] > best_split_point

        left_X, left_y = X[left_mask], y[left_mask]
        right_X, right_y = X[right_mask], y[right_mask]

        # Recursively build the left and right subtrees
        node['left'] = self._build_tree(left_X, left_y, depth + 1)
        node['right'] = self._build_tree(right_X, right_y, depth + 1)

        return node

    # method to compute the score based on the criteria
    def _compute_score(self, y):
        # Compute the score based on the selected criterion
        if self.criterion == 'gini':
            return self._gini_index(y)
        elif self.criterion == 'entropy':
            return self._entropy(y)
        elif self.criterion == 'log_loss':
            return self._log_loss(y)
        else:
            raise ValueError("Invalid criterion: {}".format(self.criterion))

    # method that makes the input(features) traverse the tree and comeback as results(target), makes a list of predicted results
    def predict(self, X):
        # Make predictions for the input data
        predictions = np.zeros(len(X))

        for i, sample in enumerate(X):
            predictions[i] = self._traverse_tree(sample, self.tree)

        return predictions

    # method to traverse the tree
    def _traverse_tree(self, sample, node):
        if 'class' in node:
            # Leaf node: return the predicted class
            return node['class']

        # Internal node: recursively traverse the tree
        feature = node['feature']
        split_point = node['split_point']

        if sample[feature] <= split_point:
            return self._traverse_tree(sample, node['left'])
        else:
            return self._traverse_tree(sample, node['right'])

    @staticmethod
    def _gini_index(y):
        # Compute the Gini index for a given set of labels
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1.0 - np.sum(probabilities ** 2)
        return gini

    @staticmethod
    def _entropy(y):
        # Compute the entropy for a given set of labels
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    @staticmethod
    def _log_loss(y):
        # Compute the logarithmic loss for a given set of labels
        num_samples = len(y)
        unique_classes = np.unique(y)

        class_counts = Counter(y)
        class_probabilities = np.array([class_counts[c] / num_samples for c in unique_classes])

        return -np.sum(class_probabilities * np.log2(class_probabilities))

    # method to assist building the tree
    def _get_best_split(self, X, y):
        best_feature = None
        best_split_point = None
        best_score = -np.inf

        for feature in range(X.shape[1]):
            unique_values = np.unique(X[:, feature])
            for value in unique_values:
                left_mask = X[:, feature] <= value
                right_mask = X[:, feature] > value
                left_labels = y[left_mask]
                right_labels = y[right_mask]

                if len(left_labels) > 0 and len(right_labels) > 0:
                    score = self._compute_score(left_labels) * len(left_labels) + \
                            self._compute_score(right_labels) * len(right_labels)
                    if score > best_score:
                        best_feature = feature
                        best_split_point = value
                        best_score = score

        return best_feature, best_split_point

    # method to gest the best criterion based on the scored from a trained tree
    def get_best_criterion(self, X, y, criteria):
        best_score = 0.0
        best_criterion = None

        for criterion in criteria:
            self.criterion = criterion
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
            self.fit(X_train, y_train)
            y_pred = self.predict(X_test)
            score = accuracy_score(y_test, y_pred)

            if score > best_score:
                best_score = score
                best_criterion = criterion

        return best_criterion

In [19]:
# tests in each tree for the forest which criterion is the best
class Custom_RF_all(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.estimators = []

    # trains the forest, in each tree it decided the best criterion individually
    def fit(self, X, y):
        self.estimators = []
        # self.max_depth = self._get_best_max_depth(X, y) # sets the max depth to its optimal value (takes too long, but it works)
        # self.n_estimators = self._get_best_n_estimators(X, y) # sets the n_estimators to its optimal value (takes too long, but it works)

        criteria = ["gini", "entropy", "log_loss"]
        used_criterion=[]

        for _ in range(self.n_estimators):
            bootstrap_indices = resample(range(len(X)), replace=True)
            X_bootstrap = X[bootstrap_indices]
            y_bootstrap = y[bootstrap_indices]

            tree = DecisionTree(self.max_depth)
            best_criterion=tree.get_best_criterion(X_bootstrap, y_bootstrap, criteria)
            used_criterion.append(best_criterion)

            tree.criterion=best_criterion
            tree.fit(X_bootstrap, y_bootstrap)

            self.estimators.append(tree)

        print("Most used criterion in our implementation: '"+ str(Counter(used_criterion).most_common(1)[0][0])+ "'")


    # method that makes the input(features) traverse the tree and comeback as results(target), makes a list of predicted results
    def predict(self, X):
        predictions = np.zeros((len(X), len(self.estimators)))

        for i, estimator in enumerate(self.estimators):
            predictions[:, i] = estimator.predict(X)

        return np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predictions)

    @staticmethod
    def _get_best_max_depth(X, y):
        param_grid = {'max_depth': [None, 5, 10, 15, 20]}
        rf_classifier = Custom_RF(n_estimators=1)  # Temporarily create an instance for grid search
        grid_search = GridSearchCV(rf_classifier, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X, y)
        best_max_depth = grid_search.best_params_['max_depth']
        return best_max_depth

    def _get_best_n_estimators(self, X, y):
        param_grid = {'n_estimators': [50, 100, 150, 200, 250]}
        rf_classifier = Custom_RF(max_depth=self.max_depth)  # Temporarily create an instance for grid search
        grid_search = GridSearchCV(rf_classifier, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X, y)
        best_n_estimators = grid_search.best_params_['n_estimators']
        return best_n_estimators

In [20]:
# tests the algorithm with the 3 criteria and returns the best one
class Custom_RF(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.estimators = []

    def get_accuracy(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

    # trains the forest with the same criterion all the way and in the end it chooses the forest with best score
    def fit(self, X, y):
        self.estimators = []
        # self.max_depth = self._get_best_max_depth(X, y) # sets the max depth to its optimal value (takes too long, but it works)
        # self.n_estimators = self._get_best_n_estimators(X, y) # sets the n_estimators to its optimal value (takes too long, but it works)

        criteria = ["gini", "entropy", "log_loss"]
        best_score = 0.0
        best_criterion = None

        for criterion in criteria:
            temp_rf = Custom_RF(self.n_estimators, self.max_depth)
            for _ in range(self.n_estimators):
                tree = DecisionTree(self.max_depth)
                tree.criterion=criterion
                bootstrap_indices = resample(range(len(X)), replace=True)
                X_bootstrap = X[bootstrap_indices]
                y_bootstrap = y[bootstrap_indices]

                tree.fit(X_bootstrap, y_bootstrap)

                temp_rf.estimators.append(tree)

            score = temp_rf.get_accuracy(X, y)

            if  score > best_score:
                best_score = score
                best_criterion = criterion
                self.estimators = temp_rf.estimators

        print("Highest scored criterion in our implementation: '"+ best_criterion+ "'")

    # method that makes the input(features) traverse the tree and comeback as results(target), makes a list of predicted results
    def predict(self, X):
        predictions = np.zeros((len(X), len(self.estimators)))

        for i, estimator in enumerate(self.estimators):
            predictions[:, i] = estimator.predict(X)

        return np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predictions)

    @staticmethod
    def _get_best_max_depth(X, y):
        param_grid = {'max_depth': [None, 5, 10, 15, 20]}
        rf_classifier = Custom_RF(n_estimators=1)  # Temporarily create an instance for grid search
        grid_search = GridSearchCV(rf_classifier, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X, y)
        best_max_depth = grid_search.best_params_['max_depth']
        return best_max_depth

    def _get_best_n_estimators(self, X, y):
        param_grid = {'n_estimators': [50, 100, 150, 200, 250]}
        rf_classifier = Custom_RF(max_depth=self.max_depth)  # Temporarily create an instance for grid search
        grid_search = GridSearchCV(rf_classifier, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X, y)
        best_n_estimators = grid_search.best_params_['n_estimators']
        return best_n_estimators

## Testes para cada dataset

In [21]:
def test_model(dataset):
    def test(model):
        X=dataset.iloc[:,1:-1].values
        y=dataset.iloc[:,-1].values

        X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42)

        model.fit(X_train, y_train)

        print("Accuracy: "+ str(round(accuracy_score(y_test, model.predict(X_test)),7)))


    print("Random Forest Classifier from sklearn using 'gini' criterion:")
    test(RF())

    print("\nRandom Forest Classifier from sklearn using 'entropy' criterion:")
    test(RF(criterion='entropy'))

    print("\nRandom Forest Classifier from sklearn using 'log_loss' criterion:")
    test(RF(criterion='log_loss'))

    print("\nOur model of Random Forest Classifier:")
    ti=time()
    test(Custom_RF())
    tf=time()
    t=tf-ti
    print("Time: "+str(int(t//60))+" minutes and "+str(round(t%60,2))+" seconds")

    print("\nOur model of Random Forest Classifier but each tree is choosing the best criterion:")
    ti=time()
    test(Custom_RF_all())
    tf=time()
    t=tf-ti
    print("Time: "+str(int(t//60))+" minutes and "+str(round(t%60,2))+" seconds")


### DATASET 1 - Cancro da mama

In [49]:
df = pd.read_csv('breast.csv')
df.head()

,id,Clump_Thickness,Cell_Size_Uniformity,Cell_Shape_Uniformity,Marginal_Adhesion,Single_Epi_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1,5,1,1,1,2,1,3,1,1,benign
1,2,5,4,4,5,7,10,3,2,1,benign
2,3,3,1,1,1,2,2,3,1,1,benign
3,4,6,8,8,1,3,4,3,7,1,benign
4,5,4,1,1,3,2,1,3,1,1,benign


In [50]:
df['Class']=df['Class'].map({'benign': 0, 'malignant': 1})
df['Bare_Nuclei'].replace('?', np.nan, inplace=True)

In [51]:
df.isnull().sum()

id                        0
Clump_Thickness           0
Cell_Size_Uniformity      0
Cell_Shape_Uniformity     0
Marginal_Adhesion         0
Single_Epi_Cell_Size      0
Bare_Nuclei              16
Bland_Chromatin           0
Normal_Nucleoli           0
Mitoses                   0
Class                     0
dtype: int64

In [52]:
df.dropna(subset=['Bare_Nuclei'], inplace=True)
df.head()

,id,Clump_Thickness,Cell_Size_Uniformity,Cell_Shape_Uniformity,Marginal_Adhesion,Single_Epi_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1,5,1,1,1,2,1,3,1,1,0
1,2,5,4,4,5,7,10,3,2,1,0
2,3,3,1,1,1,2,2,3,1,1,0
3,4,6,8,8,1,3,4,3,7,1,0
4,5,4,1,1,3,2,1,3,1,1,0


In [53]:
df.isnull().sum()

id                       0
Clump_Thickness          0
Cell_Size_Uniformity     0
Cell_Shape_Uniformity    0
Marginal_Adhesion        0
Single_Epi_Cell_Size     0
Bare_Nuclei              0
Bland_Chromatin          0
Normal_Nucleoli          0
Mitoses                  0
Class                    0
dtype: int64

In [54]:
df.describe()

,id,Clump_Thickness,Cell_Size_Uniformity,Cell_Shape_Uniformity,Marginal_Adhesion,Single_Epi_Cell_Size,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
count,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,352.355783,4.442167,3.150805,3.215227,2.830161,3.234261,3.445095,2.869693,1.603221,0.349927
std,202.563927,2.820761,3.065145,2.988581,2.864562,2.223085,2.449697,3.052666,1.732674,0.477296
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,177.500000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,0.000000
50%,356.000000,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,0.000000
75%,527.500000,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,1.000000
max,699.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000


#### Tests

In [24]:
test_model(df)

Random Forest Classifier from sklearn using 'gini' criterion:
Accuracy: 0.9609756

Random Forest Classifier from sklearn using 'entropy' criterion:
Accuracy: 0.9560976

Random Forest Classifier from sklearn using 'log_loss' criterion:
Accuracy: 0.9609756

Our model of Random Forest Classifier:
Highest scored criterion in our implementation: 'gini'
Accuracy: 0.9658537
Time: 0 minutes and 42.53 seconds

Our model of Random Forest Classifier but each tree is choosing the best criterion:
Most used criterion in our implementation: 'gini'
Accuracy: 0.9609756
Time: 0 minutes and 43.75 seconds


### Dataset 2 - Biodeg

In [35]:
df = pd.read_csv('biodeg.csv')
df.head()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V33,V34,V35,V36,V37,V38,V39,V40,V41,Class
0,1,3.919,2.6909,0,0,0,0,0,31.4,2,...,0,0,0,2.949,1.591,0,7.253,0,0,2
1,2,4.170,2.1144,0,0,0,0,0,30.8,1,...,0,0,0,3.315,1.967,0,7.257,0,0,2
2,3,3.932,3.2512,0,0,0,0,0,26.7,2,...,0,0,1,3.076,2.417,0,7.601,0,0,2
3,4,3.000,2.7098,0,0,0,0,0,20.0,0,...,0,0,1,3.046,5.000,0,6.690,0,0,2
4,5,4.236,3.3944,0,0,0,0,0,29.4,2,...,0,0,0,3.351,2.405,0,8.003,0,0,2


In [36]:
df.isnull().sum()

id       0
V1       0
V2       0
V3       0
V4       0
V5       0
V6       0
V7       0
V8       0
V9       0
V10      0
V11      0
V12      0
V13      0
V14      0
V15      0
V16      0
V17      0
V18      0
V19      0
V20      0
V21      0
V22      0
V23      0
V24      0
V25      0
V26      0
V27      0
V28      0
V29      0
V30      0
V31      0
V32      0
V33      0
V34      0
V35      0
V36      0
V37      0
V38      0
V39      0
V40      0
V41      0
Class    0
dtype: int64

In [37]:
df.describe()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V33,V34,V35,V36,V37,V38,V39,V40,V41,Class
count,1055.00000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,...,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000,1055.000000
mean,528.00000,4.782644,3.069508,0.716588,0.042654,0.980095,0.290047,1.646445,37.055640,1.376303,...,0.883412,1.274882,0.961137,3.918240,2.558417,0.686256,8.629492,0.051185,0.723223,1.337441
std,304.69657,0.546916,0.831308,1.462452,0.256010,2.332955,1.073771,2.224822,9.144466,1.963521,...,1.520467,2.273994,1.257013,0.999602,0.642765,1.090389,1.241986,0.318970,2.239286,0.473061
min,1.00000,2.000000,0.803900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.267000,1.467000,0.000000,4.917000,0.000000,0.000000,1.000000
25%,264.50000,4.481000,2.502750,0.000000,0.000000,0.000000,0.000000,0.000000,30.450000,0.000000,...,0.000000,0.000000,0.000000,3.487500,2.103000,0.000000,7.991000,0.000000,0.000000,1.000000
50%,528.00000,4.828000,3.046300,0.000000,0.000000,0.000000,0.000000,1.000000,37.500000,1.000000,...,0.000000,0.000000,1.000000,3.726000,2.458000,0.000000,8.499000,0.000000,0.000000,1.000000
75%,791.50000,5.125000,3.437650,1.000000,0.000000,1.000000,0.000000,3.000000,43.400000,2.000000,...,1.000000,2.000000,2.000000,3.987000,2.870500,1.000000,9.020500,0.000000,0.000000,2.000000
max,1055.00000,6.496000,9.177500,12.000000,3.000000,36.000000,13.000000,18.000000,60.700000,24.000000,...,12.000000,18.000000,7.000000,10.695000,5.825000,8.000000,14.700000,4.000000,27.000000,2.000000


#### Tests

In [26]:
test_model(df)

Random Forest Classifier from sklearn using 'gini' criterion:
Accuracy: 0.8675079

Random Forest Classifier from sklearn using 'entropy' criterion:
Accuracy: 0.8706625

Random Forest Classifier from sklearn using 'log_loss' criterion:
Accuracy: 0.8832808

Our model of Random Forest Classifier:
Highest scored criterion in our implementation: 'gini'
Accuracy: 0.7066246
Time: 10 minutes and 54.85 seconds

Our model of Random Forest Classifier but each tree is choosing the best criterion:
Most used criterion in our implementation: 'gini'
Accuracy: 0.7066246
Time: 10 minutes and 51.29 seconds


### Dataset 3 - Diabetes

In [45]:
df = pd.read_csv('diabetes.csv')
df.head()

,id,'preg','plas','pres','skin','insu','mass','pedi','age','class'
0,1,6,148,72,35,0,33.6,0.627,50,tested_positive
1,2,1,85,66,29,0,26.6,0.351,31,tested_negative
2,3,8,183,64,0,0,23.3,0.672,32,tested_positive
3,4,1,89,66,23,94,28.1,0.167,21,tested_negative
4,5,0,137,40,35,168,43.1,2.288,33,tested_positive


In [46]:
df["'class'"]=df["'class'"].map({'tested_positive': 0, 'tested_negative': 1})
df.head()

,id,'preg','plas','pres','skin','insu','mass','pedi','age','class'
0,1,6,148,72,35,0,33.6,0.627,50,0
1,2,1,85,66,29,0,26.6,0.351,31,1
2,3,8,183,64,0,0,23.3,0.672,32,0
3,4,1,89,66,23,94,28.1,0.167,21,1
4,5,0,137,40,35,168,43.1,2.288,33,0


In [47]:
df.isnull().sum()

id         0
'preg'     0
'plas'     0
'pres'     0
'skin'     0
'insu'     0
'mass'     0
'pedi'     0
'age'      0
'class'    0
dtype: int64

In [48]:
df.describe()

,id,'preg','plas','pres','skin','insu','mass','pedi','age','class'
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,384.500000,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.651042
std,221.846794,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,192.750000,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,384.500000,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,1.000000
75%,576.250000,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,768.000000,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


#### Tests

In [29]:
test_model(df)

Random Forest Classifier from sklearn using 'gini' criterion:
Accuracy: 0.7489177

Random Forest Classifier from sklearn using 'entropy' criterion:
Accuracy: 0.7272727

Random Forest Classifier from sklearn using 'log_loss' criterion:
Accuracy: 0.7575758

Our model of Random Forest Classifier:
Highest scored criterion in our implementation: 'gini'
Accuracy: 0.6623377
Time: 2 minutes and 14.8 seconds

Our model of Random Forest Classifier but each tree is choosing the best criterion:
Most used criterion in our implementation: 'gini'
Accuracy: 0.6580087
Time: 2 minutes and 18.35 seconds
